1) Import libraries

In [5]:
import pandas as pd

2) Import the layoff data

In [15]:
# Read the CSV file for New York layoffs
df_ny = pd.read_csv('../data/tech_layoffs_ny.csv')
# Display the first few rows
print("New York Layoffs:") 
df_ny.head()


New York Layoffs:


,State,Company,City,Number of Workers,WARN Received Date,Effective Date,Closure/Layoff,Temporary/Permanent,Union,Region,County,Industry,Notes
0,New York,Victoria's Secret (5th Avenue store & Herald S...,New York,740.0,08/31/2020,04/05/2020,Temporary Plant Closing,NaN,NaN,New York City,New York,Retail store,Unforeseeable business circumstances prompted ...
1,New York,"Tapestry, Inc.",New York,290.0,08/31/2020,03/13/2020,Temporary Plant Closing/Plant Layoff,NaN,NaN,New York City,New York,American multinational luxury fashion holding ...,Unforeseeable business circumstances prompted ...
2,New York,"Galaxy Restaurants Catering Group, LP aka Cons...",Bronx,210.0,08/31/2020,03/16/2020,Temporary Plant Closing/Plant Layoff,NaN,NaN,New York City,Bronx,Restaurant,Unforeseeable business circumstances prompted ...
3,New York,Access - Supports for Living Inc. (at U.S. Mer...,Kings Point,53.0,08/28/2020,11/30/2020,Plant Closing,NaN,NaN,Long Island,Nassau,Service academy in Nassau County,Termination of Access’s service contract with ...
4,New York,Aramark (at Pricewaterhouse Coopers),New York,35.0,08/28/2020,03/25/2020,Temporary Plant Closing/Plant Layoff,NaN,NaN,New York City,New York,Food Services,Unforeseeable business circumstances prompted ...


In [16]:
# Read the CSV file for Texas layoffs
df_te = pd.read_csv('../data/tech_layoffs_te.csv')
# Display the first few rows
print("Texas Layoffs:")
df_te.head()

Texas Layoffs:


,State,Company,City,Number of Workers,WARN Received Date,Effective Date,Closure/Layoff,Temporary/Permanent,Union,Region,County,Industry,Notes
0,Texas,XiFin,Austin,87,06/04/2025,05/29/2025,NaN,NaN,NaN,Capital Area WDA,Travis,NaN,NaN
1,Texas,Eckerd Connects (David Carrasco Job Corps),El Paso,77,06/04/2025,06/13/2025,NaN,NaN,NaN,Borderplexjobs WDA,El Paso,NaN,NaN
2,Texas,Sunnova Energy Corporation,Houston,NaN,06/03/2025,05/30/2025,NaN,NaN,NaN,Gulf Coast WDA,Harris,NaN,NaN
3,Texas,JC Penney (Allaince Supply Chain),Haslet,296,06/03/2025,08/01/2025,NaN,NaN,NaN,Tarrant County WDA,Tarrant,NaN,NaN
4,Texas,Texana Center (Building A) Airport Service Center,Rosenberg,27,05/28/2025,05/21/2025,NaN,NaN,NaN,Gulf Coast WDA,Fort Bend,NaN,NaN


3) Filter to keep only the big tech companies

In [ ]:
# We are just gonna keep the big tech companies: Amazon, Apple, Google, Meta, Microsoft, Tesla and Twitter
